In [106]:
import pandas as pd
import numpy as np

from xgboost import XGBClassifier
from feature_engine.selection import DropConstantFeatures
from sklearn.feature_selection import mutual_info_classif
from sklearn.metrics import mutual_info_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder

#### Reading the data

In [7]:
fraud_df = pd.read_csv("fraud_detection.csv")


#### Basic EDA

In [28]:
# fraud_df.head()
# fraud_df.shape
fraud_df.columns

Index(['transaction_id', 'user_id', 'transaction_amount',
       'transaction_date_and_time', 'merchant_id', 'payment_method',
       'country_code', 'transaction_type', 'device_type', 'ip_address',
       'browser_type', 'operating_system', 'merchant_category', 'user_age',
       'user_occupation', 'user_income', 'user_gender', 'user_account_status',
       'transaction_status', 'location_distance', 'time_taken_for_transaction',
       'transaction_time_of_day', 'user's_transaction_history',
       'merchant's_reputation_score', 'user's_device_location',
       'transaction_currency', 'transaction_purpose', 'user's_credit_score',
       'user's_email_domain', 'merchant's_business_age',
       'transaction_authentication_method', 'fraudulent_flag'],
      dtype='object')

In [29]:
features = ["transaction_amount", "payment_method", "country_code", "transaction_type",
 "device_type", "ip_address", "browser_type", "operating_system", "merchant_category",
            "user_age", "user_occupation", "user_income", "user_gender", 'user_account_status',
            'transaction_status', 'location_distance', 'time_taken_for_transaction',
            'transaction_time_of_day', 'user\'s_transaction_history',
            'merchant\'s_reputation_score', 'user\'s_device_location',
            'transaction_currency', 'transaction_purpose', 'user\'s_credit_score',
            'user\'s_email_domain', 'merchant\'s_business_age',
            'transaction_authentication_method']


In [82]:
# formatting the column names to be more consistent
fraud_df.columns = fraud_df.columns.str.lower()
fraud_df.columns = fraud_df.columns.str.replace(" ", "_")

### Feature Engineering

In [85]:
for feature in features:
    score = mutual_info_score(fraud_df[feature], y)
    print(f"{feature} score: {score}")


c:\Users\Latitude E7270\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\cluster\_supervised.py:64: UserWarning: Clustering metrics expects discrete values but received continuous values for label, and binary values for target
  warnings.warn(msg, UserWarning)


transaction_amount score: 0.008438973328968282
payment_method score: 2.5987580564157398e-06
country_code score: 4.687493972477252e-06
transaction_type score: 3.3740052077374516e-06
device_type score: 3.241385290793375e-06
ip_address score: 0.6926499624915344
browser_type score: 2.8168583201398922e-06
operating_system score: 3.859630139907422e-06
merchant_category score: 4.300995773337535e-06
user_age score: 3.5765949986665824e-06
user_occupation score: 1.7762668802490111e-06


c:\Users\Latitude E7270\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\cluster\_supervised.py:64: UserWarning: Clustering metrics expects discrete values but received continuous values for label, and binary values for target
  warnings.warn(msg, UserWarning)


user_income score: 0.5175025935456573
user_gender score: 4.111489797231638e-07
user_account_status score: 1.1851414232627588e-06
transaction_status score: 2.7601645790423235e-06


c:\Users\Latitude E7270\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\cluster\_supervised.py:64: UserWarning: Clustering metrics expects discrete values but received continuous values for label, and binary values for target
  warnings.warn(msg, UserWarning)


location_distance score: 0.0008197739407848333


c:\Users\Latitude E7270\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\cluster\_supervised.py:64: UserWarning: Clustering metrics expects discrete values but received continuous values for label, and binary values for target
  warnings.warn(msg, UserWarning)


time_taken_for_transaction score: 0.00048606497360688356
transaction_time_of_day score: 1.1841384206512373e-07
user's_transaction_history score: 7.72316001679263e-06


c:\Users\Latitude E7270\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\cluster\_supervised.py:64: UserWarning: Clustering metrics expects discrete values but received continuous values for label, and binary values for target
  warnings.warn(msg, UserWarning)


merchant's_reputation_score score: 3.270236507881951e-05
user's_device_location score: 2.3434770907021507e-06
transaction_currency score: 3.881151857003384e-06
transaction_purpose score: 3.0466044109236923e-06
user's_credit_score score: 4.9009240722788956e-05
user's_email_domain score: 3.736723703025302e-06
merchant's_business_age score: 1.9052730870217083e-06
transaction_authentication_method score: 2.8808243364569774e-06


In [100]:
selected_features = ["user_income", "transaction_amount",
                     "location_distance", "time_taken_for_transaction",
                     "user's_credit_score", "merchant_category", "merchant's_reputation_score",
                     "transaction_time_of_day", "user's_credit_score", "transaction_authentication_method"]

# seperating the data into target (y) and predictor (x) variables.
y = fraud_df["fraudulent_flag"]
X = fraud_df[selected_features]

### Encoding categorical variables

In [136]:
encoder = LabelEncoder()
merchant = encoder.fit_transform(fraud_df["merchant_category"])
time = encoder.fit_transform(fraud_df["transaction_time_of_day"])
auth = encoder.fit_transform(fraud_df["transaction_authentication_method"])

0          22
1           4
2          29
3           0
4          23
           ..
5999995    14
5999996     0
5999997    31
5999998     3
5999999     4
Name: merchant, Length: 6000000, dtype: int32

In [143]:
merchant = pd.DataFrame(merchant)
time = pd.DataFrame(time)
auth = pd.DataFrame(auth)

cat_col = pd.concat([merchant, time, auth], axis=1)
cat_col= pd.concat([cat_col.iloc[:, 0].rename("merchant", inplace=True), 
           cat_col.iloc[:, 1].rename("time", inplace=True),
           cat_col.iloc[:, 2].rename("auth", inplace=True)], axis=1)



In [147]:
# dropping the categorical variables
X.drop(["merchant_category",
        "transaction_time_of_day",
        "transaction_authentication_method"], axis=1,
       inplace=True)

# adding the encoded variables

C:\Users\Latitude E7270\AppData\Local\Temp\ipykernel_12984\1948458391.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X.drop(["merchant_category",


#### Splitting the dataset into train and test

In [101]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [102]:
X_train.reset_index(drop=True, inplace=True)

In [103]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)
X_train.head()

(4200000, 10)
(1800000, 10)
(4200000,)
(1800000,)


,user_income,transaction_amount,location_distance,time_taken_for_transaction,user's_credit_score,merchant_category,merchant's_reputation_score,transaction_time_of_day,user's_credit_score,transaction_authentication_method
0,21748.03,3.33,14.55,28.16,401,Industrial & Scientific,4.56,Nighttime,401,Time-Based OTP
1,5008.78,777.10,7.48,14.69,598,Electronics Repair,3.46,Evening,598,CAPTCHA
2,15057.28,592.86,2.00,28.52,436,Industrial & Scientific,4.50,Evening,436,Two-Factor Authentication
3,64460.29,902.30,91.15,49.28,444,Food & Beverage,3.26,Evening,444,Hardware Token
4,83425.41,359.44,54.78,59.26,536,Office Supplies,1.84,Evening,536,Push Notification Confirmation


### Model selection and fitting

In [104]:
XGmodel = XGBClassifier(n_estimators=2, max_depth=2,
                        learning_rate=1, objective='binary:logistic')

XGmodel.fit(X_train, y_train)

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "c:\Users\Latitude E7270\AppData\Local\Programs\Python\Python311\Lib\site-packages\IPython\core\interactiveshell.py", line 3508, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\Latitude E7270\AppData\Local\Temp\ipykernel_12984\3289419558.py", line 4, in <module>
    XGmodel.fit(X_train, y_train)
  File "c:\Users\Latitude E7270\AppData\Local\Programs\Python\Python311\Lib\site-packages\xgboost\core.py", line 620, in inner_f
    ):
      
  File "c:\Users\Latitude E7270\AppData\Local\Programs\Python\Python311\Lib\site-packages\xgboost\sklearn.py", line 1471, in fit
  File "c:\Users\Latitude E7270\AppData\Local\Programs\Python\Python311\Lib\site-packages\xgboost\sklearn.py", line 448, in _wrap_evaluation_matrices
  File "c:\Users\Latitude E7270\AppData\Local\Programs\Python\Python311\Lib\site-packages\xgboost\sklearn.py", line 908, in _create_dmatrix
    from_fit = True
       ^^^^^^^^^^^^
  File "c:\Users\Latitud

In [105]:
y_pred = XGmodel.predict(X_test)

accuracy_score(y_pred, y_test)

NotFittedError: need to call fit or load_model beforehand

In [ ]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.50      0.05      0.09    900156
           1       0.50      0.95      0.66    899844

    accuracy                           0.50   1800000
   macro avg       0.50      0.50      0.37   1800000
weighted avg       0.50      0.50      0.37   1800000

